In [ ]:
import requests
import os
import importlib
import pandas as pd
import numpy as np
%matplotlib inline
from yelpapi import YelpAPI
import pdb
import json

# Load Yelp data

In [ ]:
working_dir = 'C:\\Users\\Chauncey\\Documents\\GitHub\\Kibbeh'
#working_dir = 'C:\\Users\\Me\\Documents\\GitHub\\Kibbeh'
with open(working_dir+ '\\yelp_api_key.txt', 'r') as api_file:
    api_key =  api_file.read()
    [CONSUMER_KEY, CONSUMER_SECRET, TOKEN, TOKEN_SECRET] = api_key.split('\n')

In [ ]:
yelp_api = YelpAPI(CONSUMER_KEY, CONSUMER_SECRET, TOKEN, TOKEN_SECRET)

In [ ]:
def get_restaurants_yelp():
    restaurant_data = []
    for cur_offset in range(0, 1000, 20):
        # sort = 0 sets by default, limited to 1000, sort = 2 is ranked from best to worst, but limited to 40
        cur_query = yelp_api.search_query(term='food', location = 'Seattle', sort=0, limit=20, offset = cur_offset) 
        #pdb.set_trace()
        restaurant_data.extend( cur_query['businesses'])
    return restaurant_data

In [ ]:
restaurant_json = get_restaurants()
len(restaurant_json)

In [ ]:
# save
with open('restaurant_list.json', 'w') as json_file:
    json.dump(restaurant_json, json_file)

In [ ]:
# load
with open('restaurant_list.json', 'r') as json_file:
    restaurant_json = json.load( json_file)

# Load Foursquare data

In [ ]:
working_dir = 'C:\\Users\\Chauncey\\Documents\\GitHub\\Kibbeh'
#working_dir = 'C:\\Users\\Me\\Documents\\GitHub\\Kibbeh'
with open(working_dir+ '\\foursquare_api_key.txt', 'r') as api_file:
    api_key =  api_file.read()
    [FOURSQUARE_ID, FOURSQUARE_SECRET] = api_key.split('\n')

In [ ]:
import foursquare

In [ ]:
foursquare_client = foursquare.Foursquare(FOURSQUARE_ID, FOURSQUARE_SECRET)

In [ ]:
def get_restaurants_4square():
    restaurant_data = []
    offset_delta = 50
    for cur_offset in range(0, 500, offset_delta):
        foursquare_params = {'near':'Seattle, WA', 'section':'food', 
                             'limit':offset_delta, 'offset':cur_offset}
        cur_query = foursquare_client.venues.explore(params=foursquare_params)
        #pdb.set_trace()
        restaurant_data.extend( cur_query['groups'][0]['items'])
    return restaurant_data

In [ ]:
fs_test = foursquare_client.venues.explore(params={'near':'Seattle, WA', 'section':'food'})
# fs json has a list of stuff, but only one element
# each element has a name, items, and type, but I only care about the items
# items is a list of places; each place has keys: referralId, reasons, tips, and venue
# the venue key has the detailed information

In [ ]:
len(fs_test['groups'][0]['items'])

In [135]:
fs_test2 = get_restaurants_4square()

> <ipython-input-134-e6f29d769534>(9)get_restaurants_4square()
-> restaurant_data.extend( cur_query['groups'][0]['items'])
(Pdb) cur_query
{'totalResults': 250, 'headerLocationGranularity': 'city', 'headerFullLocation': 'Seattle', 'query': 'food', 'groups': [{'name': 'recommended', 'items': [{'referralId': 'e-3-49ba3a34f964a5205f531fe3-0', 'reasons': {'count': 0, 'items': [{'reasonName': 'globalInteractionReason', 'summary': 'This spot is popular', 'type': 'general'}]}, 'tips': [{'url': 'http://seattletimes.nwsource.com/html/outdoors/2008785702_nwwmicrohood26.html', 'text': "Easily one of Seattle's best bakeries, especially for baguettes and croissants. Recent addition: deep-dish pizza. Grab a croissant or two. That alone will entice you back to this neighborhood again.", 'logView': True, 'canonicalUrl': 'https://foursquare.com/item/4bf1cdde70c603bb04e79bb4', 'todo': {'count': 20}, 'user': {'id': '1150073', 'firstName': 'The Seattle Times', 'photo': {'prefix': 'https://irs2.4sqi.net/im

BdbQuit: 

In [ ]:
len(fs_test2)

# Load OpenMenu data

# Feature extraction

In [ ]:
# yelp feature extraction
def feature_extract_list(json_list):
    #restaurant_df = pd.DataFrame(columns = ['rating', 'categories', 'name', 'phone', 'review_count'])
    return pd.concat(map(feature_extract_json, json_list))

def feature_extract_json(cur_json):
    #print(cur_json['name'])
    feature_list = ('id', 'rating', 'categories', 'name', 'phone', 'review_count')
    cur_dict = dict((x, cur_json[x]) for x in feature_list if x in cur_json.keys())
    cur_dict['categories'] = [[x[0] for x in cur_dict['categories']]]
    return pd.DataFrame(cur_dict).set_index('name')

In [ ]:
test = test_load[0]
test

In [ ]:
ganma = feature_extract_list(restaurant_json)
ganma.head()

In [ ]:
pd.scatter_matrix(ganma);

In [ ]:
yelp_api.business_query(restaurant_json[20]['id'])